## https://flightaware.com/live/fleet/LAN


In [ ]:
from robobrowser import RoboBrowser
import plotly.plotly as py
import pandas as pd
import networkx as nx
from cachier import cachier
import geocoder
browser = RoboBrowser(history=True, parser='html5lib')

In [ ]:
url = 'https://flightaware.com/live/fleet/LAN'
browser.open(url)
table=browser.select('.prettyTable')[0]
table_code=table.__str__()
table_code[:1000]

In [ ]:
tdf=pd.read_html(table_code, header=0)[0]
tdf

In [ ]:
import re
pattern = r'<tr><th class="mainHeader" colspan="8">.*?</th></tr>'
table_code = re.sub(pattern, "", table_code)
table_code

In [ ]:
tdf=pd.read_html(table_code, header=0)[0]
tdf

## http://flightaware.com/live/fleet/AAL?;offset=20

In [ ]:
browser = RoboBrowser(history=True, parser='html5lib')
base_url = 'https://flightaware.com/live/fleet/'
query = 'JAL' ## 
airlines = "LAN", "JAL", "DLH", "AAL", "CCA", 'AFR', 'IGO', 'MSR'
df = pd.DataFrame()

for query in airlines:
    print('-----------')
    print(query)
    for i in range(2):
        url = base_url + query + "?;offset=" + str(i*20)
        browser.open(url)
        table=browser.select('.prettyTable')[0]
        t=table.__str__()
        if 'Sorry' in t:
                break
        pattern = r'<tr><th class="mainHeader" colspan="8">.*?</th></tr>'
        t = re.sub(pattern, "", t)
        tdf=pd.read_html(t, header=0)[0]
        df = df.append(tdf)
        print(tdf.head(3))

In [ ]:
df.head(10)

In [ ]:
from joblib import Memory
memory = Memory(cachedir='tmp', verbose=0)

@memory.cache
def get_latlng(query):
    g=geocoder.google(query)
    if g:
        return g.latlng
    else:
        return None, None

In [ ]:

great_lines = []
airports = []
airport_data = []
seen=[]
great_lines_2=[]
airports = pd.DataFrame(columns=['label','lat','lon','size'])

    
i=0
for index,row in df.iterrows():
    origin = row.Origin
    destination = row.Destination
    olat, olon = get_latlng(origin)
    dlat, dlon = get_latlng(destination)
    if olat and dlat:
        if origin not in seen:
            seen.append(origin)
            print(origin, olat, olon)
        if destination not in seen:
            seen.append(destination)
            print(destination, dlat, dlon)
        great_lines.append(dict(
            type = 'scattergeo',
            lat = [ olat, dlat ],
            lon = [ olon, dlon ],
            hoverinfo = 'Ident',
                text = 'Ident',
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'rgba(255,0,0,0.2)',
                ),
                )
        )
        airports.loc[i]=[origin, olat, olon, 0.0]
        airports.loc[i+1]= [destination, dlat, dlon, 0.0]
        i+=2
     

In [ ]:
great_lines

In [ ]:
airports.head(20)

In [ ]:
df.head()

## Grafo
![Graph](graph.png)

In [ ]:
import networkx as nx
G = nx.from_pandas_dataframe(df, 'Origin', 'Destination')

In [ ]:
for node in G.nodes():
    print(node)

In [ ]:
for edge in G.edges():
    print(edge)

## Betweenness Centrality
![Betweenness](betweenness.png)

In [ ]:
centrality=nx.betweenness_centrality(G)
centrality

In [ ]:
for key, item in centrality.items():
    airports.loc[airports.label==key, 'size'] = (item*100)+10
airports.head(20)

## https://plot.ly/

In [ ]:
import plotly.plotly as py

airport_markers = [ dict(
        type = 'scattergeo',
        lon = airports['lon'],
        lat = airports['lat'],
        hoverinfo = 'label',
        text = airports['label'],
        mode = 'markers',
        marker = dict( 
            size=airports['size'], 
            color='red',   
        ))]
    
layout = dict(
        title = 'Current flight network',
        showlegend = False,  
        width = 800,
        height = 800,
        geo = dict(
            showland = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = '#fff',
            oceancolor = '#eee',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            
        )
    )
    
fig = dict( data=great_lines+airport_markers, layout=layout )
py.iplot( fig, validate=False, filename='d3-globe' )